In [7]:
#correct things are imported
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import pchip_interpolate

from genetic_algorithm_pfm.genetic_algorithm_pfm.algorithm import GeneticAlgorithm

In [8]:
# x1 = area in m2
# x2 = distance from shore
# x3 = eco friendly building cost

c1 = 25   # Costs per material m3
c2 = 4   # Cost of fuel use per L
c3 = 0.1   # Emission of materials per m3
c4 = 1.2   # Emission of fuel per L
c5 = 150000 # Capacity per m2 in kW / m2
c6 = 1/10 # Scaling factor feasible distance
c7 = 1 #still has to be defined
c8 = 1 #still has to be defined
c9 = 1 #still has to be defined
c10 = 1 #still has to be defined

k1 = 1.7   # Height of island
k2 = 1.9   # Ship size
k3 = 0.0009   # fuel use per m

constants_list = [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, k1, k2, k3]

def F1(x1, x2, x3, constants_list=constants_list): #this function calculates the total material use in m3
    F1_completed = x1 * constants_list[10]
    return F1_completed

def F2(x1, x2, x3, constants_list=constants_list): #this function calculates the total fuel use in L
    F1_run = F1(x1, x2, x3, constants_list)
    F2_completed = constants_list[12] * ((F1_run / constants_list[11]) *x2)
    return F2_run

In [9]:
def objective_p1(x1, x2, x3, constants_list):
    """
    Objective to minimize the costs
    x1: size of the island (in m2)
    x2: distance to the mainland (in m)
    x3: cost of ecofriendly building
    Result: the cost of building the island in euro
    """
    F1 = F1(x1, x2, x3, constants_list) # Material in m3
    F2 = F2(x1, x2, x3, constants_list) # Fuel in L
    
    O1 = F1 * c1 + F2 * c2 + x3
    return pchip_interpolate([1, 10, 50], [100, 20, 0], O1)
#the pchip interpolate has wrong values, these need to be changed!!!!!!!!!

In [10]:
def objective_p2(x1, x2, x3, constants_list=constants_list):
    """
    Objective to minimize the emissions.
    x1: size of the island (in m2)
    x2: distance to the mainland (in m)

    Result: total emissions of co2 in kg
    """
    F1 = F1(x1, x2, x3, constants_list) # Material in m3
    F2 = F2(x1, x2, x3, constants_list) # Fuel in L

    O2 = F1 * c3 + F2 * c4
    return pchip_interpolate([0.5, 4, 10], [100, 80, 0], O2)
#the pchip interpolate has wrong values, these need to be changed!!!!!!!!!

In [11]:
def objective_p3(x1, x2, x3, constants_list=constants_list):
    """
    Objective to minimize the emissions.
    x1: size of the island (in m2)
    x2: distance to the mainland (in m)

    Result: amount of W that the island is able to produce
    """

    O3 = x1 * c5 * (np.exp(-c6 * x2) + 1)

    return pchip_interpolate([0, 1, 5 , 8.5], [0, 0, 100 ,100], O3)
#the pchip interpolate has wrong values, these need to be changed!!!!!!!!!

In [12]:
def objective(variables):
    """
    Objective function that is fed to the GA. Calles the separate preference functions that are declared above.

    :param variables: array with design variable values per member of the population. Can be split by using array
    slicing
    :return: 1D-array with aggregated preference scores for the members of the population.
    """
    # extract 1D design variable arrays from full 'variables' array
    x1 = variables[:, 0]
    x2 = variables[:, 1]
    x3 = variables[:, 2]

    # calculate the preference scores
    p_1 = objective_p1(x1, x2, x3)
    p_2 = objective_p2(x1, x2, x3)
    p_3 = objective_p3(x1, x2, x3)

    # aggregate preference scores and return this to the GA
    return [0.34, 0.33, 0.33], [p_1, p_2, p_3]

In [13]:
#fuctions for constraints
cons = []

b1 = [5, 10000]  # x1 bounds
b2 = [5, 40000]  # x2 bounds
b3 = [0, 1]  # x3 bounds
#b4 bound for x4
bounds = [b1, b2, b3]

In [14]:
#two ways of optimising results
paradigm = ['minmax', 'tetra']
marker = ['o', '*']
colours = ['orange','green']

for i in range(2):
    # make dictionary with parameter settings for the GA run with the IMAP solver
    options = {
        'n_bits': 8,
        'n_iter': 400,
        'n_pop': 500,
        'r_cross': 0.8,
        'max_stall': 8,
        'aggregation': paradigm[i], # minmax or tetra
        'var_type': 'real'
    }

    # run the GA and print its result
    print(f'Run GA with ' + paradigm[i])
    ga = GeneticAlgorithm(objective=objective, constraints=cons, bounds=bounds, options=options)
    score_IMAP, design_variables_IMAP, _ = ga.run()

Run GA with minmax
The type of aggregation is set to minmax
Generation   Best score   Mean             Max stall    Diversity    Number of non-feasible results
8
5
243
256
10000


ValueError: X2 > 7000. Je script is kapot!